In [1]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt
from math import pi

import numpy as np
from numpy import random

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


In [152]:
latent_dim = 5
n_outputs = 1
model = Sequential()
model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim, name = "1stlayer"))
model.add(Dense(n_outputs, activation='linear', name ="2dlayer"))


In [153]:
def pm0(x, m , s ):
    dist = tfp.distributions.Normal(m, s)
    return (np.sqrt(2*pi)*s)*dist.prob(x)

m_data =  2
s_data =  1
cte_init = 1/(np.sqrt(2*pi)*s_data)

cte = cte_init

batch_size = 100
#def custom_loss(layer):

latent_dim =  5

x_input = randn(latent_dim * batch_size)
# reshape into a batch of inputs for the network
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer

placeholder_pm = 1
placeholder_pn = 1
placeholder_ratio = 1
def nce_loss(y_true,y_pred):
        
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
   
    p_n = pn.prob( y_pred) 
    tf.print(" shape of y", tf.shape(y_pred))
    tf.print(" shape of pm", tf.shape(pm0(y_pred, m_data, s_data)))
    p_m = tf.reshape(pm0(y_pred, m_data, s_data), [batch_size])# noise (generator) density 
    temp = tf.math.reduce_mean( p_m)
    temp1 = tf.math.reduce_mean( p_n)
    proba = p_n/(cte*p_m+p_n)
    indexes =  tf.where(tf.math.is_nan(p_n/(cte*p_m+p_n)))
    indexes_inf =  tf.where(tf.math.is_inf(p_n/(cte*p_m+p_n)))
    argmin_pn = (p_n)
    argmin_pm = tf.math.argmin(p_m)
        
    tf.print ("confusion p_m" , temp)
    tf.print ("confusion p_n" , temp1)
    tf.print (" indexes nan " , indexes)
    tf.print (" indexes inf " , indexes_inf)
    tf.print(" shape" ,indexes.shape)
    tf.print("type" , type(indexes.shape[0]))
    #tf.print("size", tf.size(indexes).eval())
    
    
    
    try:
        
        tf.print("Hello try boolean mask " )
        #tf.print("mask min pn en fonction de pm" ,tf.boolean_mask(p_n,  tf.math.equal(p_n , tf.math.reduce_min(p_m))))
        tf.print("mask min pn en fonction de pm",tf.gather(p_n,tf.math.argmin(p_m)))
        tf.print(" shape of pm", tf.shape(p_m))
        tf.print(" shape of pn", tf.shape(p_n))
        tf.print(" shape of y", tf.shape(y_pred))
        tf.print("shape of ratio2", tf.shape(tf.math.divide(p_n,tf.math.add(cte*p_m,p_n))))
        tf.print("mask min ratio en fonction de pm",tf.gather(p_n/(cte*p_m+p_n),tf.math.argmin(p_m)))
        
        
        #tf.print("mask nan pn ",tf.where(p_n,tf.math.is_nan(p_n/(cte*p_m+p_n))))
        #tf.print("mask nan", tf.boolean_mask(p_n, tf.math.is_nan(p_n/(cte*p_m+p_n))))
    
        
        #tf.print("condition for if ",tf.size(indexes).numpy()!=0)
        if (tf.size(indexes).eval()!=0):
        #if indexes.shape[0] >0:
            proba_nan= tf.gather(p_n, indexes[0])
            pn_inf= tf.gather(p_n, indexes_inf[0])
            tf.print ("pn where its nan " , proba_nan)
            tf.print ("pn where its inf " , pn_inf)
    except Exception as e:
        tf.print("\n !!!!! " ,e)
        pass

    tf.print("minimum de pm ",tf.math.reduce_min(p_m))
    tf.print("minimum de pn ",tf.math.reduce_min(p_n))
    
    
    tf.print("argmin de pm ",tf.math.argmin(p_m))
    tf.print("argmin de pn ",tf.math.argmin(p_n))
    
    result = tf.math.reduce_mean( tf.math.log(tf.math.divide(p_n,(cte*p_m+p_n))))
    #result = tf.math.reduce_mean(p_m)
    #print("log likelihood 1", result)
    tf.print("\n log likelihood 2", result)
    tf.print("likelihood 2", tf.math.reduce_mean( (p_n/(cte*p_m+p_n))))
    
    placeholder_pm = p_m
    placeholder_pn = p_n
    placeholder_ratio = p_n/(cte*p_m+p_n)
    
    return result


    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def nce_metric(y_predit):
        
    tf.print("***NCE METRIC***")
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
    
    p_n = pn.prob( y_predit) 
    p_m = pm0(y_predit, m_data, s_data) # noise (generator) density 
    temp = tf.math.reduce_mean( p_m)
    temp1 = tf.math.reduce_mean( p_n)
    proba = p_n/(cte*p_m+p_n)
    indexes =  tf.where(tf.math.is_nan(p_n/(cte*p_m+p_n)))
    indexes_inf =  tf.where(tf.math.is_inf(p_n/(cte*p_m+p_n)))
    
#     tf.print ("confusion p_m" , temp)
#     tf.print ("confusion p_n" , temp1)
#     tf.print (" indexes nan " , indexes)
#     tf.print (" indexes inf " , indexes_inf)
#     tf.print(" shape" ,indexes.shape)
#     tf.print("type" , type(indexes.shape[0]))
#     tf.print("size", tf.size(indexes))
    
    #tf.print(tf.size(indexes).numpy())
    if (tf.size(indexes).numpy()!=0):
        #if indexes.shape[0] >0:
        proba_nan= tf.gather(p_n, indexes[0][0])
        pn_inf= tf.gather(p_n, indexes_inf[0])
        tf.print ("pn where its nan " , proba_nan)
        tf.print ("pn where its inf " , pn_inf)

    
    #print(np.mean(y_predit))
    p_n = pn.prob( y_predit) 
    p_m = pm0(y_predit, m_data, s_data) # noise (generator) density 
    #print("pn ", np.mean(p_n))
    #print("p_m ", np.mean(p_m))
    temp = tf.math.reduce_mean( (p_n/(cte*p_m+p_n)))
    tf.print ("****confusion proba nce metric " , temp)
    return temp

In [154]:
model = Sequential()
model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim, name = "1stlayer"))
model.add(Dense(n_outputs, activation='linear', name ="2dlayer"))


model.compile(loss=nce_loss, optimizer='adam')


In [155]:
y_train = np.ones(batch_size)
y_predit = model.predict (x_train)
model.train_on_batch(x_train, y_train )
nce_metric(y_predit)

 shape of y [100 1]
 shape of pm [100 1]
confusion p_m 0.229565009
confusion p_n 0.313157082
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0605931096
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.997387469

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  0.000397844327
minimum de pn  0.0377223678
argmin de pm  36
argmin de pn  26

 log likelihood 2 -0.27295506
likelihood 2 0.80161345
***NCE METRIC***
****confusion proba nce metric  0.795334876


<tf.Tensor: shape=(), dtype=float32, numpy=0.7953349>

In [141]:
placeholder_pm

1

In [135]:
n_epochs = 500
for i in range(n_epochs):

    x_input = randn(latent_dim * batch_size)
    # reshape into a batch of inputs for the network
    x_train = x_input.reshape(batch_size, latent_dim)
    y_train = np.ones(batch_size)
    y_predit = model.predict (x_train)
    
    print( " ############ ", i , " ###############")
    print(  " y BEFORE train ", np.mean(y_predit))
    #nce_metric(y_predit)
    
    with tf.GradientTape() as tape:
        loss = nce_loss(y_train, model(x_train))

    #print(tape.gradient(loss, model.trainable_weights))
    
    print("x shape", x_train.shape)
    print("y shape", y_train.shape)
    #tf.print(" model evaluate " ,model.evaluate(x_train, y_train))
    #print(" y a til des nan before" ,np.sum(np.isnan(model.get_weights()[0])))
    nce_metric(y_predit)
    tf.print(" model train on batch " , model.train_on_batch(x_train, y_train ))
    
    y_predit = model.predict (x_train)
    
    print( "y after train ", np.mean(y_predit))
    
    #print(" y a til des nan after" ,np.sum(np.isnan(model.get_weights()[0])))
    
    
    
    #print(" variance " , np.std(y_predit))
   
    
    if np.sum(np.isnan(model.get_weights()[0])) != 0:
        break

 ############  0  ###############
 y BEFORE train  -0.993119
confusion p_m 0.181971848
confusion p_n 0.12760365
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0187213626
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  1.62320944e-17
minimum de pn  0.0187213626
argmin de pm  8
argmin de pn  8

 log likelihood 2 -0.332422018
likelihood 2 0.781159282
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.76475
confusion p_m 0.181971848
confusion p_n 0.137521148
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0124469688
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
s

 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  1.1390756e-12
minimum de pn  0.0225092769
argmin de pm  77
argmin de pn  97

 log likelihood 2 -0.394898295
likelihood 2 0.752656341
 model train on batch  -0.3948983
y after train  -0.73546356
 ############  6  ###############
 y BEFORE train  -1.066058
confusion p_m 0.141300738
confusion p_n 0.135343418
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.025353631
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  2.298

minimum de pm  1.27357349e-36
minimum de pn  0.00216598809
argmin de pm  28
argmin de pn  28

 log likelihood 2 -0.31642276
likelihood 2 0.790638268
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.77768904
confusion p_m 0.161548212
confusion p_n 0.138485581
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.000113052323
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  1.27355403e-36
minimum de pn  0.000113052323
argmin de pm  28
argmin de pn  28

 log likelihood 2 -0.3104904
likelihood 2 0.797267318
 model train on batch  -0.3104904
y after train  -1.039691
 ############  12  ###############
 y BEFORE train

 ############  17  ###############
 y BEFORE train  -0.52343374
confusion p_m 0.241011009
confusion p_n 0.137686312
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0157493055
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  9.13376837e-14
minimum de pn  0.0157493055
argmin de pm  43
argmin de pn  43

 log likelihood 2 -0.419897497
likelihood 2 0.735340595
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.726330101
confusion p_m 0.241011009
confusion p_n 0.138010085
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0243703891
 shape of pm [100]
 shape of pn [100]
 shape of y

 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  5.42797381e-14
minimum de pn  0.0233937372
argmin de pm  74
argmin de pn  74

 log likelihood 2 -0.352041781
likelihood 2 0.768854141
 model train on batch  -0.35204178
y after train  -0.6867379
 ############  23  ###############
 y BEFORE train  -0.4487534
confusion p_m 0.22559613
confusion p_n 0.133217052
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.00996367354
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 1

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?

...

minimum de pm  0.0798835382
minimum de pn  0.0113256071
argmin de pm  99
argmin de pn  99

 log likelihood 2 -1.91002107
likelihood 2 0.150940195
 model train on batch  -1.9100211
y after train  2.5580106
 ############  473  ###############
 y BEFORE train  2.5878155
confusion p_m 0.779948294
confusion p_n 0.442285389
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.173317105
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.651927829

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  0.231953591
minimum de pn  0.132928029
argmin de pm  90
argmin de pn  49

 log likelihood 2 -0.535621285
likelihood 2 0.590123594
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.585994601
confusion p_m 0.779948354
confusion p_n 0.05225

****confusion proba nce metric  0.540175378
confusion p_m 0.74580431
confusion p_n 0.0532620288
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0076958579
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.371059746

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  0.0326973461
minimum de pn  0.0076958579
argmin de pm  22
argmin de pn  22

 log likelihood 2 -1.88620031
likelihood 2 0.156177938
 model train on batch  -1.8862003
y after train  2.6049378
 ############  479  ###############
 y BEFORE train  2.5275707
confusion p_m 0.79419595
confusion p_n 0.436992049
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction 

 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0696775764
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.724620461

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  0.0663749278
minimum de pn  0.0696775764
argmin de pm  80
argmin de pn  80

 log likelihood 2 -0.543803036
likelihood 2 0.585893869
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.577672541
confusion p_m 0.747860432
confusion p_n 0.0497674569
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0104172323
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.282332808

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No de

mask min ratio en fonction de pm 0.271769017

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  0.0730380192
minimum de pn  0.0108740125
argmin de pm  79
argmin de pn  79

 log likelihood 2 -1.93296778
likelihood 2 0.146835804
 model train on batch  -1.9329678
y after train  2.6348314
 ############  490  ###############
 y BEFORE train  2.6685815
confusion p_m 0.738957644
confusion p_n 0.373596638
 indexes nan  []
 indexes inf  []
 shape TensorShape([0, 1])
type <class 'int'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0586812831
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.768857062

 !!!!!  NotImplementedError("eval is not supported when eager execution is enabled, is .numpy() what you're looking for?")
minimum de pm  0.0442205928
minimum de pn  0.0586812831
argmin

minimum de pm  0.013277581
minimum de pn  0.0452445596
argmin de pm  89
argmin de pn  89

 log likelihood 2 -0.560004354
likelihood 2 0.583083332
x shape (100, 5)
y shape (100,)
***NCE METRIC***
****confusion proba nce metric  0.570309758
confusion p_m 0.67595613
confusion p_n 0.045613233
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0053929179
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.504487276

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  0.0132775623
minimum de pn  0.0053929179
argmin de pm  89
argmin de pn  89

 log likelihood 2 -1.89579868
likelihood 2 0.155525252
 model train on batch  -1.8957987
y after train  2.8256278
 ############  496  ###############
 y BEFORE tr

In [159]:
tf.print(" model evaluate " ,model.evaluate(x_train, y_train,batch_size= batch_size) )

 shape of y [100 1]
 shape of pm [100 1]
confusion p_m 0.232798189
confusion p_n 0.313285559
 indexes nan  []
 indexes inf  []
 shape TensorShape([None, 1])
type <class 'NoneType'>
Hello try boolean mask 
mask min pn en fonction de pm 0.0621398501
 shape of pm [100]
 shape of pn [100]
 shape of y [100 1]
shape of ratio2 [100]
mask min ratio en fonction de pm 0.9973104

 !!!!!  ValueError('Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`')
minimum de pm  0.000420062308
minimum de pn  0.0363907814
argmin de pm  36
argmin de pn  26

 log likelihood 2 -0.277942389
likelihood 2 0.79883033
100/100 [==============================] - 0s 3ms/sample - loss: -0.2779
 model evaluate  -0.27794238924980164


In [200]:
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
# Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
nce_metric(y_predit)

confusion proba  tf.Tensor(0.9876856, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=0.9876856>

In [158]:
y_train

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [91]:
y_predit = model.predict (x_train)
print(np.mean(y_predit))

-1.8780168


In [92]:
pm0(-1.8780168, m_data,s_data)

<tf.Tensor: shape=(), dtype=float32, numpy=0.00054239994>

In [121]:
with tf.GradientTape() as tape:
    loss = nce_loss(y_train, model(x_train))

tape.gradient(loss, model.trainable_weights)

confusion  tf.Tensor(nan, shape=(), dtype=float32)


[<tf.Tensor: shape=(5, 15), dtype=float32, numpy=
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan]], dtype=float32)>,
 <tf.Tensor: shape=(15,), dtype=float32, numpy=
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan], dtype=float32)>,
 <tf.Tensor: shape=(15, 1), dtype=float32, numpy=
 array([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], dtype=float32)>,
 <tf.Tensor:

In [190]:
tensor = tf.range(10)
tf.print(tensor)

[0 1 2 ... 7 8 9]


In [214]:
0.0311287586/0.041129888

0.7568403444230142

In [215]:
cte

0.3989422804014327

In [62]:
cte

0.3989422804014327